<a href="https://colab.research.google.com/github/iiney/colabbb/blob/main/easy%E5%AD%B8%E6%AC%8Achatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl

In [ ]:
import google.generativeai as genai
import os

# 設API密鑰:看你們要不要弄一個自己的API 還是用我的 我都可以(但我有限制說只能用生成式AI
api_key =
url = f"https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={api_key}"

In [ ]:
#檢查 api 是否 work!!要記得跑喔 有出現 "1. Bebop\n2. Big Band\n3. Cool Jazz\n4. Fusion\n5. Smooth Jazz" 之類的 = 成功

import requests
import json

headers = {
    'Content-Type': 'application/json',
}

data = {
    "contents": [
        {
            "role": "user",
            "parts": [
                {"text": "Give me five subcategories of jazz?"}
            ]
        }
    ]
}

# Make the API request
response = requests.post(url, headers=headers, json=data)

# Pretty-print the JSON response
response_json = response.json()
print(json.dumps(response_json, indent=4))


{
    "candidates": [
        {
            "content": {
                "parts": [
                    {
                        "text": "1. Bebop\n2. Swing\n3. Cool Jazz\n4. Fusion\n5. Avant-Garde"
                    }
                ],
                "role": "model"
            },
            "finishReason": "STOP",
            "index": 0,
            "safetyRatings": [
                {
                    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                    "probability": "NEGLIGIBLE"
                },
                {
                    "category": "HARM_CATEGORY_HATE_SPEECH",
                    "probability": "NEGLIGIBLE"
                },
                {
                    "category": "HARM_CATEGORY_HARASSMENT",
                    "probability": "NEGLIGIBLE"
                },
                {
                    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                    "probability": "NEGLIGIBLE"
                }
            ]
        }
   

In [ ]:
import re
import nltk
import requests

# 初始化NLTK
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#設定 gemini生成的東西!!!要記得跑喔

def generate_gemini_content(prompt):

    # Define the request data
    data = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }

    # Define the headers
    headers = {
        'Content-Type': 'application/json',
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the generated content from the response JSON
        generated_content = response.json()['candidates'][0]['content']

        # Extract and return the summary for each paragraph
        return [generated_content]
    else:
        # If the request was not successful, print an error message and return None
        print("Error:", response.status_code, response.text)
        return None

In [ ]:
import pandas as pd
from google.colab import files

def simplify_question(original_question):
    # Generate Gemini response
    response = generate_gemini_content(
        prompt=f"請幫助簡化以下提問，使其更具體清晰，並只以問題回復，：{original_question}")
    # Extract the simplified question from the response
    simplified_question = response[0]['parts'][0]['text'].strip()

    return simplified_question

def ask_student_confirmation(simplified_question):
    print(f"這是我們簡化後的問題：\n\n{simplified_question}\n\n這符合您的原意嗎？")
    return

def simplify_question_2():
    global previous_result
    new_question = input("請重述您的問題以讓我們更了解:")
    # Generate Gemini response
    response = generate_gemini_content(
        prompt=f"請幫助簡化以下提問，使其更具體清晰，並只以問題回復，：{new_question}")

    # Extract the simplified question from the response
    simplifie_question_2 = response[0]['parts'][0]['text'].strip()
    previous_result = simplifie_question_2
    return simplifie_question_2

def ask_student_confirmation_2(simplified_question_2):
    print(f"這是我們簡化後的問題：\n\n{simplified_question_2}\n\n這符合您的原意嗎？")
    return

def reask_question():
    simplified_question_2=simplify_question_2()
    ask_student_confirmation_2(simplified_question_2)
    return

def check_info():
  global final_question, student_id, class_name, name
  print(f"請問以下資訊是否正確? 系及班級: {class_name}, 學號: {student_id}, 姓名: {name}， 問題:{final_question}")
  return

def save_to_excel(class_name, student_id, name, final_question, filename='student_info.xlsx'):
    # Create a DataFrame with the data
    data = {
        '系及班級': [class_name],
        '學號': [student_id],
        '姓名': [name],
        '問題': [final_question]
    }
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"資料已保存至 {filename}")

def main( ):

  global previous_result, final_question, student_id, class_name, name

  student_id = input("請輸入學號: ")
  class_name = input("請輸入系及班級: ")
  name = input("請輸入姓名: ")

  original_question = input("請輸入您的問題：")


  simplified_question = simplify_question(original_question)
  ask_student_confirmation(simplified_question)

  continue_loop_1 = True
  counter = 0
  final_question = None

  while continue_loop_1:
      # Get user's response
      answer_one = input("請回答[是]或[否]：")

      if answer_one == "是":
        if counter == 0:
          # Exit the loop if the response is "是"
          final_question = simplified_question
          continue_loop_1 = False

        else:
          final_question = previous_result
          continue_loop_1 = False

      elif answer_one == "否":
          counter += 1

          # If the counter reaches 5, print "問題仍未能符合原意，請聯繫支持。" and exit the loop
          if counter == 5:
              print("問題仍未能符合原意，請聯繫支持。")
              break
          else:
            simplified_question = reask_question()

      else:
          # If the response is neither "是" nor "否", print "Invalid answer" and prompt again
          print("無效答案。請以 '是' 或 '否' 回覆")

  print("問題已確認。")

  continue_loop_2 = True

  while continue_loop_2:
    check_info()
    answer_two = input("請回答[是]或[否]：")

    if answer_two == "是":
      print("已成功保存並準備處理您的問題")
      continue_loop_2 = False
    elif answer_two == "否":
      print("資訊輸入錯誤，重新載入程式")
      main()
      return

    else:
      print("無效答案。請以 '是' 或 '否' 回覆")

  filename = 'student_info.xlsx'
  save_to_excel(class_name, student_id, name, final_question, filename)
  files.download(filename)
  print("資料已保存至 student_info.xlsx，檔案以下載")

  return

if __name__ == "__main__":
    main()


請輸入學號: 110830017
請輸入系及班級: 化工三甲
請輸入姓名: 楊宜軒
請輸入您的問題：烏龜
這是我們簡化後的問題：

烏龜的哪些方面需要具體說明或澄清？

這符合您的原意嗎？
請回答[是]或[否]：否
請重述您的問題以讓我們更了解:學校有幾隻烏龜
這是我們簡化後的問題：

學校有幾隻烏龜？

這符合您的原意嗎？
請回答[是]或[否]：對
無效答案。請以 '是' 或 '否' 回覆
請回答[是]或[否]：是
問題已確認。
請問以下資訊是否正確? 系及班級: 化工三甲, 學號: 110830017, 姓名: 楊宜軒， 問題:學校有幾隻烏龜？
請回答[是]或[否]：對
無效答案。請以 '是' 或 '否' 回覆
請問以下資訊是否正確? 系及班級: 化工三甲, 學號: 110830017, 姓名: 楊宜軒， 問題:學校有幾隻烏龜？
請回答[是]或[否]：是
已成功保存並準備處理您的問題
資料已保存至 student_info.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

資料已保存至 student_info.xlsx，檔案以下載
